In [ ]:
from DeepFMKit.experiments import Experiment
from DeepFMKit.waveforms import second_harmonic_distortion
from DeepFMKit.plotting import default_rc

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from functools import partial
import matplotlib.colors as mcolors

plt.rcParams.update(default_rc)

print("DeepFMKit modules loaded successfully.")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Pandas version: {pd.__version__}")

In [ ]:
m = 6.0

from DeepFMKit.factories import StandardDFMIExperimentFactory
"""
def __call__(self, params: dict) -> dict:
    m_main = params['m_main']
    waveform_kwargs = params.get('waveform_kwargs', {})
    laser_config = physics.LaserConfig()
    main_ifo_config = physics.InterferometerConfig(label="main_ifo")
    main_ifo_config.ref_arml = 0.1
    main_ifo_config.meas_arml = main_ifo_config.ref_arml + self.opd_main
    laser_config.waveform_func = self.waveform_func_to_use
    laser_config.waveform_kwargs = waveform_kwargs
    laser_config.df = (m_main * sc.c) / (2 * np.pi * self.opd_main)
"""
# Load my custom experiment factory:
factory = StandardDFMIExperimentFactory(
    waveform_function=second_harmonic_distortion,
    opd_main=0.1
)

# Create Experiment object:
experiment = Experiment(description="2nd Harmonic Distortion")
experiment.set_config_factory(factory)

# Set Experiment parameters:
experiment.n_trials = 500 # 50 trials per nominal_amplitude point for good statistics
experiment.f_samp = 200e3 # Sampling frequency
experiment.n_fit_buffers_per_trial = 1 # Modulation cycles per fit cycle

# Define sweep axis:
axis = np.linspace(0.0, 0.02, 10)
experiment.add_axis('distortion_amp', axis)

# Static parameters for all trials:
experiment.set_static({
    'm_main': m,                    # Keep modulation depth constant
})

# Variables to Monte-Carlo over:
experiment.add_stochastic_variable(
    'waveform_kwargs', 
    lambda dist_amp: {'distortion_amp': dist_amp, 'distortion_phase': np.random.uniform(0, 2*np.pi)},
    depends_on='distortion_amp'
)

experiment.add_stochastic_variable('phi', lambda: np.random.uniform(0, 2*np.pi))

# Analysis to perform:
experiment.add_analysis(
    name='NLS_Fit',
    fitter_method='nls',
    result_cols=['m'], # Parameters I want to collect
    fitter_kwargs={
        'ndata': 30,   # Use this many harmonics for NLS
    }
)

# Run the experiment:
print(f"Starting experiment: '{experiment.description}'...")
experiment.results = experiment.run()
print("Experiment completed.")

In [ ]:
results = experiment.results
x_axis = results['axes']['distortion_amp']

fig1, ax = plt.subplots(figsize=(3.375, 2), dpi=150)
mean_m = results['NLS_Fit']['m']['mean']
std_m = results['NLS_Fit']['m']['std']

ax.errorbar(x_axis*100, mean_m, yerr=std_m, fmt='o-', lw=2, capsize=3, markersize=5, c='k', zorder=-1)
ax.plot(x_axis*100, m*(1-x_axis), ls='--', c='red', lw=1, zorder=1)
ax.set_xlabel('Distortion amplitude (%)')
ax.set_ylabel(r'Estimate of $m$')
ax.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
# Load my custom experiment factory:
factory = StandardDFMIExperimentFactory(
    waveform_function=second_harmonic_distortion,
    opd_main=0.1
)

# Create Experiment object:
experiment = Experiment(description="2nd Harmonic Distortion")
experiment.set_config_factory(factory)

# Set Experiment parameters:
experiment.n_trials = 20 # 50 trials per nominal_amplitude point for good statistics
experiment.f_samp = 200e3 # Sampling frequency
experiment.n_fit_buffers_per_trial = 1 # Modulation cycles per fit cycle

# Define sweep axis:
experiment.add_axis('m_main', np.linspace(3.0, 30.0, 30))
experiment.add_axis('distortion_amp', np.linspace(0.0, 0.2, 30))

# Variables to Monte-Carlo over:
experiment.add_stochastic_variable(
    'waveform_kwargs', 
    lambda dist_amp: {'distortion_amp': dist_amp, 'distortion_phase': np.random.uniform(0, 2*np.pi)},
    depends_on='distortion_amp'
)

experiment.add_stochastic_variable('phi', lambda: np.random.uniform(0, 2*np.pi))

# Analysis to perform:
experiment.add_analysis(
    name='NLS_Fit',
    fitter_method='nls',
    result_cols=['m'], # Parameters I want to collect
    fitter_kwargs={
        'ndata': 50,   # Use this many harmonics for NLS
    }
)

# Run the experiment:
print(f"Starting experiment: '{experiment.description}'...")
experiment.results = experiment.run()
print("Experiment completed.")

In [ ]:
results = experiment.results
threshold = 3.0

# --- 1. Extract Axes and Data ---
m_true_axis = results['axes']['m_main']
distortion_amp_axis = results['axes']['distortion_amp']
m_estimated_all = results['NLS_Fit']['m']['all_trials']

# --- 2. Calculate Bias ---
# I need to broadcast the true 'm' values to subtract them from the estimated values.
# The shape of m_estimated_all is (len(m_true_axis), len(distortion_amp_axis), n_trials).
# I'll reshape m_true_axis to (len(m_true_axis), 1, 1) for broadcasting.
m_true_broadcast = m_true_axis[:, np.newaxis, np.newaxis]
bias_all_trials = m_estimated_all - m_true_broadcast

# --- 3. Calculate Statistics for Each Panel ---
# For the mean bias, I'll take the absolute value for visualization on a single-tone colormap.
mean_bias = np.abs(np.mean(bias_all_trials, axis=-1))

# The worst-case bias is the maximum absolute deviation observed in the trials.
worst_case_bias = np.max(np.abs(bias_all_trials), axis=-1)

# --- 4. Create the Plot ---
fig, (ax1, ax2) = plt.subplots(
    1, 2,
    figsize=(6.875,2.5),
    dpi=150,
    sharey=True
)

# I'll create a custom white-to-blue colormap as requested.
custom_cmap = mcolors.LinearSegmentedColormap.from_list(
    'custom_blue', [(0, 'white'), (1, 'darkred')]
)

# --- Panel 1: Mean Bias ---
# Data must be transposed because pcolormesh(X, Y, C) expects C to have shape (len(Y), len(X)).
pcm1 = ax1.pcolormesh(
    m_true_axis,
    distortion_amp_axis * 100, # Convert to percentage for the y-axis
    mean_bias.T,
    cmap=custom_cmap,
    shading='nearest',
    vmin=0,
    vmax=threshold
)
ax1.set_title(r'Mean bias: $|E\,[\hat{m}] - m|$')
ax1.set_xlabel('True modulation depth $m$ (rad)')
ax1.set_ylabel('Distortion amplitude (%)')

# --- Panel 2: Worst-Case Bias ---
pcm2 = ax2.pcolormesh(
    m_true_axis,
    distortion_amp_axis * 100,
    worst_case_bias.T,
    cmap=custom_cmap,
    shading='nearest',
    vmin=0,
    vmax=threshold
)
ax2.set_title(r'Worst-case bias: max$| \hat{m} - m|$')
ax2.set_xlabel('True modulation depth, $m$ (rad)')

# --- Add a single Colorbar for both plots ---
fig.subplots_adjust(right=0.85) # Make space for the colorbar
cbar_ax = fig.add_axes([0.87, 0.15, 0.03, 0.7])
cbar = fig.colorbar(pcm2, cax=cbar_ax)
cbar.set_label('Bias in $m$ (rad)')

plt.show()

In [ ]:
threshold = 0.3
# --- 1. Extract Axes and Data ---
# The axes and estimated data are extracted as before.
m_true_axis = results['axes']['m_main']
distortion_amp_axis = results['axes']['distortion_amp']
m_estimated_all = results['NLS_Fit']['m']['all_trials']

m_true_broadcast = m_true_axis[:, np.newaxis, np.newaxis]
absolute_bias_all_trials = m_estimated_all - m_true_broadcast

# Now, I'll calculate the fractional bias. I use np.divide with a 'where'
# clause to robustly handle any cases where m_true might be zero,
# preventing division-by-zero warnings.
fractional_bias_all_trials = np.divide(
    absolute_bias_all_trials,
    m_true_broadcast,
    out=np.zeros_like(absolute_bias_all_trials), # Output is 0 where divisor is 0
    where=m_true_broadcast != 0
)

# For the mean fractional bias, I'll take the absolute value for visualization.
mean_fractional_bias = np.abs(np.mean(fractional_bias_all_trials, axis=-1))

# The worst-case fractional bias is the maximum absolute deviation observed.
worst_case_fractional_bias = np.max(np.abs(fractional_bias_all_trials), axis=-1)

fig, (ax1, ax2) = plt.subplots(
    1, 2,
    figsize=(6.875,2.5),
    dpi=150,
    sharey=True
)

# I will use the same custom white-to-blue colormap.
custom_cmap = mcolors.LinearSegmentedColormap.from_list(
    'custom_blue', [(0, 'white'), (1, 'darkblue')]
)

# --- Panel 1: Mean Fractional Bias ---
# Data must be transposed because pcolormesh(X, Y, C) expects C to have shape (len(Y), len(X)).
pcm1 = ax1.pcolormesh(
    m_true_axis,
    distortion_amp_axis * 100, # Convert to percentage for the y-axis
    mean_fractional_bias.T,
    cmap=custom_cmap,
    shading='nearest',
    vmin=0,
    vmax=threshold
)
ax1.set_title('Mean')
ax1.set_xlabel('True modulation depth $m$ (rad)')
ax1.set_ylabel('Distortion amplitude (%)')
ax1.grid(False)

# --- Panel 2: Worst-Case Fractional Bias ---
pcm2 = ax2.pcolormesh(
    m_true_axis,
    distortion_amp_axis * 100,
    worst_case_fractional_bias.T,
    cmap=custom_cmap,
    shading='nearest',
    vmin=0,
    vmax=threshold
)
ax2.set_title('Worst-case')
ax2.set_xlabel('True modulation depth, $m$ (rad)')
ax2.grid(False)

# --- Add a single Colorbar for both plots ---
fig.subplots_adjust(right=0.85) # Make space for the colorbar
cbar_ax = fig.add_axes([0.87, 0.15, 0.03, 0.7])
cbar = fig.colorbar(pcm2, cax=cbar_ax)
cbar.set_label('Fractional bias $|(\\hat{m}-m)/m|$')
cbar_ax.grid(False)

plt.show()